In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

24/09/25 02:29:04 WARN Utils: Your hostname, DESKTOP-AEUBGUH resolves to a loopback address: 127.0.1.1; using 172.20.208.1 instead (on interface eth1)
24/09/25 02:29:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 02:29:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pd_df = pd.read_parquet("../data/clean_data/df_comments_video.parquet")
df = spark.createDataFrame(pd_df)

In [4]:
df.show()

24/09/25 02:29:25 WARN TaskSetManager: Stage 0 contains a task of very large size (4578 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+--------------------+----------+-------------------+-------------------+---------------+-----------+--------------------+
|          comment_id|              author|   author_channel_id|                text|like_count|       published_at|         updated_at|totalReplyCount|   video_id|          clean_text|
+--------------------+--------------------+--------------------+--------------------+----------+-------------------+-------------------+---------------+-----------+--------------------+
|UgxqJ2eO2p7w2NEvB...|  @emmanuelamama1991|UCdUooL3DTt3Wj0M3...|great video,helpful.|       1.0|2024-08-17 15:07:48|2024-08-17 15:07:48|            1.0|qtlUwwtvuEg|  great videohelpful|
|UgzmXwaNzivNHqjfR...|   @dr.alexshayo6972|UCN56M0YoTn18rd_K...|Thank you for suc...|       4.0|2023-05-01 16:48:21|2023-05-01 16:48:21|            1.0|qtlUwwtvuEg|thank well explai...|
|UgyySzt4xOZ1hKdS9...|     @mohdkashif4596|UCr5Ua5AsmM6phaRu...|very u

In [5]:
from transformers import pipeline
from pyspark.sql.functions import udf

sentiment_model = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

def get_sentiment(text):
    try:
        sentiment = sentiment_model(text)
        return sentiment
    except Exception as e:
        return str(e)

sentiment_udf = udf(get_sentiment)

/home/ivanseldasp/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# df_limited = df.limit(10000)

df_with_sentiment = df.withColumn("emotions", sentiment_udf(df['translation']))

df_with_sentiment.show(truncate=False)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `translation` cannot be resolved. Did you mean one of the following? [`comment_id`, `author`, `author_channel_id`, `text`, `like_count`, `published_at`, `updated_at`, `totalReplyCount`, `video_id`, `clean_text`].

In [7]:
emotions_column = df_with_sentiment.select('emotions').limit(10)

In [8]:
emotions_column.show()

24/09/25 02:24:53 WARN TaskSetManager: Stage 2 contains a task of very large size (6407 KiB). The maximum recommended task size is 1000 KiB.
/home/ivanseldasp/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded fi

+--------------------+
|            emotions|
+--------------------+
|[[{score=0.953117...|
|[[{score=0.990995...|
|[[{score=0.988971...|
|[[{score=0.941890...|
|[[{score=0.986132...|
|[[{score=0.965435...|
|[[{score=0.984012...|
|[[{score=0.747151...|
|[[{score=0.990160...|
|[[{score=0.978443...|
+--------------------+

